**Connect google drive**

In [2]:
# Check if NVIDIA GPU is enabled
!nvidia-smi

Tue Mar 15 17:04:06 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   70C    P8    32W / 149W |      0MiB / 11441MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [3]:
from google.colab import drive
drive.mount('/content/gdrive')
# !ln -s /content/gdrive/MyDrive/mydrive
# !ln -s ./gdrive/MyDrive /mydrive
#!ln -s ./gdrive/MyDrive/mydrive
# !ls ./mydrive
# !ls ./MyDrive

Mounted at /content/gdrive
ls: cannot access './MyDrive': No such file or directory


**1) Clone the Darknet**



In [11]:
!git clone https://github.com/AlexeyAB/darknet

Cloning into 'darknet'...
remote: Enumerating objects: 15412, done.
remote: Total 15412 (delta 0), reused 0 (delta 0), pack-reused 15412
Receiving objects: 100% (15412/15412), 14.04 MiB | 18.90 MiB/s, done.
Resolving deltas: 100% (10356/10356), done.


**2) Compile Darknet using Nvidia GPU**


In [12]:
# change makefile to have GPU and OPENCV enabled
%cd darknet
!sed -i 's/OPENCV=0/OPENCV=1/' Makefile
!sed -i 's/GPU=0/GPU=1/' Makefile
!sed -i 's/CUDNN=0/CUDNN=1/' Makefile
!make

/content/darknet
mkdir -p ./obj/
mkdir -p backup
chmod +x *.sh
g++ -std=c++11 -std=c++11 -Iinclude/ -I3rdparty/stb/include -DOPENCV `pkg-config --cflags opencv4 2> /dev/null || pkg-config --cflags opencv` -DGPU -I/usr/local/cuda/include/ -DCUDNN -Wall -Wfatal-errors -Wno-unused-result -Wno-unknown-pragmas -fPIC -Ofast -DOPENCV -DGPU -DCUDNN -I/usr/local/cudnn/include -c ./src/image_opencv.cpp -o obj/image_opencv.o
./src/image_opencv.cpp: In function ‘void draw_detections_cv_v3(void**, detection*, int, float, char**, image**, int, int)’:
./src/image_opencv.cpp:946:23: warning: variable ‘rgb’ set but not used [-Wunused-but-set-variable]
                 float rgb[3];
                       ^~~
./src/image_opencv.cpp: In function ‘void draw_train_loss(char*, void**, int, float, float, int, int, float, int, char*, float, int, int, double)’:
./src/image_opencv.cpp:1147:13: warning: this ‘if’ clause does not guard... [-Wmisleading-indentation]
             if (iteration_old == 0)
           

**3) Configure Darknet network for training YOLO V3**

In [13]:
!cp cfg/yolov3.cfg cfg/yolov3_training.cfg

In [14]:
!sed -i 's/batch=1/batch=64/' cfg/yolov3_training.cfg
!sed -i 's/subdivisions=1/subdivisions=16/' cfg/yolov3_training.cfg
!sed -i 's/max_batches = 500200/max_batches = 4000/' cfg/yolov3_training.cfg
!sed -i '610 s@classes=80@classes=1@' cfg/yolov3_training.cfg
!sed -i '696 s@classes=80@classes=1@' cfg/yolov3_training.cfg
!sed -i '783 s@classes=80@classes=1@' cfg/yolov3_training.cfg
!sed -i '603 s@filters=255@filters=18@' cfg/yolov3_training.cfg
!sed -i '689 s@filters=255@filters=18@' cfg/yolov3_training.cfg
!sed -i '776 s@filters=255@filters=18@' cfg/yolov3_training.cfg

In [ ]:
# Create folder on google drive so that we can save there the weights
!mkdir "../gdrive/MyDrive/yolo_experiment"

mkdir: cannot create directory ‘../gdrive/MyDrive/yolo_experiment’: File exists


In [19]:
!echo "anta" > data/obj.names
!echo -e 'classes = 1\ntrain = data/train.txt\nvalid = data/test.txt\nnames = data/obj.names\nbackup = ../gdrive/MyDrive/yolo_experiment' > data/obj.data
!mkdir data/obj

In [20]:
# Download weights darknet model 53
!wget https://pjreddie.com/media/files/darknet53.conv.74

--2022-03-15 17:09:21--  https://pjreddie.com/media/files/darknet53.conv.74
Resolving pjreddie.com (pjreddie.com)... 128.208.4.108
Connecting to pjreddie.com (pjreddie.com)|128.208.4.108|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 162482580 (155M) [application/octet-stream]
Saving to: ‘darknet53.conv.74’

darknet53.conv.74   100%[===================>] 154.96M  47.3MB/s    in 3.5s    

2022-03-15 17:09:25 (44.2 MB/s) - ‘darknet53.conv.74’ saved [162482580/162482580]



**4) Extract Images**

The images need to be inside a zip archive called "images.zip" and they need to be inside the folder "yolov3" on Google Drive

In [21]:
!unzip "../gdrive/MyDrive/Colab Notebooks/yolo_experiment/pysource/tapirus_experiment/images.zip" -d data/obj

Archive:  ../gdrive/MyDrive/Colab Notebooks/yolo_experiment/pysource/tapirus_experiment/images.zip
  inflating: data/obj/0007 ND, Mbe Damiana MANECO (01).JPG  
  inflating: data/obj/0007 ND, Mbe Damiana MANECO (01).txt  
  inflating: data/obj/0007 ND, Mbe Damiana MANECO (02).JPG  
  inflating: data/obj/0007 ND, Mbe Damiana MANECO (02).txt  
  inflating: data/obj/0007 ND, Mbe Damiana MANECO (03).JPG  
  inflating: data/obj/0007 ND, Mbe Damiana MANECO (03).txt  
  inflating: data/obj/0008 Sergiao MANECO (01).JPG  
  inflating: data/obj/0008 Sergiao MANECO (01).txt  
  inflating: data/obj/0008 Sergiao MANECO (02).JPG  
  inflating: data/obj/0008 Sergiao MANECO (02).txt  
  inflating: data/obj/0008 Sergiao MANECO (03).JPG  
  inflating: data/obj/0008 Sergiao MANECO (03).txt  
  inflating: data/obj/0008 Sergiao MANECO (04).JPG  
  inflating: data/obj/0008 Sergiao MANECO (04).txt  
  inflating: data/obj/0008 Sergiao MANECO (05).JPG  
  inflating: data/obj/0008 Sergiao MANECO (05).txt  
  inf

In [23]:
# We're going to convert the class index on the .txt files. As we're working 
# with only one class, it's supposed to be class 0.
# If the index is different from 0 then we're going to change it.
import glob
import os
import re

txt_file_paths = glob.glob(r"data/obj/*.txt")

for i, file_path in enumerate(txt_file_paths):

    # get image size
    with open(file_path, "r") as f_o:
    
        lines = f_o.readlines()
        text_converted = []
    
        for line in lines:
      
            print('Line:', line.replace('\n', ''))
            numbers = re.findall("[0-9.]+", line)
            print('Numbers:', numbers)
      
            if numbers:

                # Define coordinates
                text = "{} {} {} {} {}".format(0, numbers[1], numbers[2], numbers[3], numbers[4])
                text_converted.append(text)
                print('Index and filepath:', i, file_path)
                print('Text:', text, '\n')

        # Write file
        with open(file_path, 'w') as fp:
    
            for item in text_converted:
      
                fp.writelines("%s\n" % item)

Line: 0 0.264757 0.451003 0.528646 0.900463
Numbers: ['0', '0.264757', '0.451003', '0.528646', '0.900463']
Index and filepath: 0 data/obj/0031 Jose PRINCIPAL (41).txt
Text: 0 0.264757 0.451003 0.528646 0.900463 

Line: 0 0.586155 0.473765 0.821615 0.939815
Numbers: ['0', '0.586155', '0.473765', '0.821615', '0.939815']
Index and filepath: 1 data/obj/0033 Maria PRINCIPAL (02).txt
Text: 0 0.586155 0.473765 0.821615 0.939815 

Line: 0 0.401042 0.542438 0.282118 0.524691
Numbers: ['0', '0.401042', '0.542438', '0.282118', '0.524691']
Index and filepath: 2 data/obj/0033 Damiana CARAGUATA (16).txt
Text: 0 0.401042 0.542438 0.282118 0.524691 

Line: 0 0.741776 0.465789 0.516447 0.738596
Numbers: ['0', '0.741776', '0.465789', '0.516447', '0.738596']
Index and filepath: 3 data/obj/0048 Sufoco CERCATAPERA3 (23).txt
Text: 0 0.741776 0.465789 0.516447 0.738596 

Line: 0 0.628947 0.345029 0.226974 0.458480
Numbers: ['0', '0.628947', '0.345029', '0.226974', '0.458480']
Index and filepath: 4 data/obj/1

In [26]:
import glob

# images_list = glob.glob("data/obj/*.jpg")
images_list = glob.glob("data/obj/*.JPG")

print(f'Number of images: {len(images_list)}')

Number of images: 435


In [27]:
#Create training.txt file
file = open("data/train.txt", "w") 
file.write("\n".join(images_list)) 
file.close() 

**6) Start the training**

In [1]:
# Start the training
!./darknet detector train data/obj.data cfg/yolov3_training.cfg darknet53.conv.74 -dont_show

/bin/bash: ./darknet: No such file or directory


~Início: 16:20 - 11/02/2022~

~Início: 15/03/2022 - 11:05h~

Início: 15/03/2022 - 14:13h
Fim: 15/03/2022 - 17:50h
